# Numerical Analysis Project
Team: 아프지마

Member

* 20181628 성창훈
* 20181635 신명철

written by 성창훈, 신명철

# Google Colab

TPOT은 NVIDIA RAPIDS 환경에서 GPU 가속을 활용할 수 있다.

그러나 Web-Based 노트북에서 해당 환경을 설치하는 것이 매우 까다롭고 오래걸려 CPU로 학습을 진행했다.

# Packages

대다수의 Web-Based notebook 서비스들(Google Colab, Kaggle, Paperspace...)은 docker를 이용해 일관된 환경을 제공한다.

환경의 업데이트는 패키지의 업데이트보다 보수적으로 이루어지므로 보통 버전이 낮다.

이때 일부 최신 패키지는 높은 버전의 pip 버전을 요하므로, 반드시 패키지 관리 도구를 최신 버전으로 업데이트하여 사용한다.

업데이트 후에는 kernel을 다시 시작한다. (초기화가 아님에 유의한다)

In [ ]:
pip install TPOT

# TPOT

TPOT은 Tree-based Pipeline Optimization Tool의 약자를 의미한다.

유전적 알고리즘을 이용해 파이프라인을 최적화하는 것을 목표로 한다.

다른 패키지와 마찬가지로 모델을 자동으로 탐색하고(NAS), 하이퍼파라메터를 최적화한다(HPO).

In [ ]:
import numpy as np
import pandas as pd
import os
from google.colab import drive
from sklearn.model_selection import train_test_split
from tpot import TPOTClassifier
from tpot import TPOTRegressor

drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


# Data Loading

Kaggle의 Leaf Rice Disease Dataset을 이용한다.

모든 노트에서 데이터 처리하는 것이 번거로우므로, ndarray화 시킨 이미지 데이터셋을 구글 드라이브에 업로드한 후, 각 노트에서 불러와 사용했다.

dataset = imgset_360_480.npy.npz

* `dataset['x'].shape : (240, 360, 480, 1)` -> Numpy ndarray image data
* `dataset['y'].shape : (240, )` -> labels composed of 0, 1, 2

TPOT 같은 경우 이미지 사이즈가 크면 메모리를 초과하여 런타임이 강제 종료되는 상황이 자주 발생했으므로, grayscale을 적용했다.

다른 패키지에 비해 입력되는 데이터의 양이 1/3 수준이다.

In [ ]:
dataset = np.load('/content/gdrive/MyDrive/imgset_360_480.npy.npz')

In [ ]:
x = dataset['x']
y = dataset['y']

In [ ]:
x = x.reshape(240,360 * 480 * 1)

In [ ]:
trainx, testx, trainy, testy = train_test_split(x, y)

# Build a classifier and Fit
TPOT이 classification에 적합한 모델을 탐색한다.

코드가 간단한 것이 AutoML의 큰 장점인데, 모델 설계와 학습을 단 두 줄에 끝냈다.

TPOT은 하드웨어 스펙의 한계로, 다른 패키지들과 비교했을 때 비교적 작은 인자를 입력했다.

유전 알고리즘에 대해서는 최대 10세대까지 진행되도록 했다.

유전 알고리즘의 출력 내용을 보면 세대에 따라 더 높은 정확성을 가지도록 모델 구조나 데이터 전처리, 경사하강 기법, 학습 속도 등을 찾아낸 것을 알 수 있다.


In [ ]:
tpot = TPOTClassifier(
  verbosity=3, 
  n_jobs=-1, 
  generations=10, 
  population_size= 1,
  early_stop = 5
)
tpot.fit(trainx, trainy)
print(tpot.score(testx, testy))


32 operators have been imported by TPOT.



Generation 1 - Current Pareto front scores:

-1	0.3333333333333333	MLPClassifier(input_matrix, MLPClassifier__alpha=0.0001, MLPClassifier__learning_rate_init=0.01)

-2	0.35	MLPClassifier(FastICA(input_matrix, FastICA__tol=0.9), MLPClassifier__alpha=0.0001, MLPClassifier__learning_rate_init=0.01)

Generation 2 - Current Pareto front scores:

-1	0.3333333333333333	MLPClassifier(input_matrix, MLPClassifier__alpha=0.0001, MLPClassifier__learning_rate_init=0.01)

-2	0.35	MLPClassifier(FastICA(input_matrix, FastICA__tol=0.9), MLPClassifier__alpha=0.0001, MLPClassifier__learning_rate_init=0.01)

Generation 3 - Current Pareto front scores:

-1	0.3333333333333333	MLPClassifier(input_matrix, MLPClassifier__alpha=0.0001, MLPClassifier__learning_rate_init=0.01)

-2	0.35	MLPClassifier(FastICA(input_matrix, FastICA__tol=0.9), MLPClassifier__alpha=0.0001, MLPClassifier__learning_rate_init=0.01)

Generation 4 - Current Pareto front scores:

-1	0.34444444444444444	MLPClassifier(input_matrix, MLPClassi

# Get accuracy of the last generation model

총 학습시간은 24분으로 30분을 학습하게 한 다른 패키지와 유사한 시간을 소요했다.

가장 마지막 세대 모델의 성능을 테스트한 결과 test_accuracy 78.3%를 달성했다.

충분한 RAM과 GPU 가속 환경이었다면 더 좋은 결과가 나왔을 것이다.





In [ ]:
print('accurate : ',tpot.score(testx,testy))

accurate :  0.7833333333333333
